In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re, json
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask
import wandb
from wandb.keras import WandbCallback


import time
print(tf.__version__)



2.4.1


In [1]:
import zipfile

with zipfile.ZipFile('/home/jovyan/git-repos/tensorflow_ds/100-bird-species.zip', 'r') as zip_ref:
    zip_ref.extractall('/home/jovyan/git-repos/tensorflow_ds/datasets/birds')

In [2]:
def get_compiled_model(base_lr):
    model = tf.keras.applications.ResNet50(
                include_top=True,
                weights=None,
                classes=10)

    optimizer = keras.optimizers.Adam(lr=base_lr)
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy']) #integer labels
    
    return model


In [7]:
def get_dataset(batch_size=32):

    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
        image_size=(224,224),
        batch_size=batch_size
    ).repeat().prefetch(1)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
        image_size=(224,224),
        batch_size=batch_size
    ).prefetch(1)
    
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/test',
        image_size=(224,224),
        batch_size=batch_size
    ).prefetch(1)
    
    return (
        train_ds, valid_ds, test_ds
    )


In [8]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))


base_lr = .02
BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset(batch_size=100)

train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
val_dist_dataset = strategy.experimental_distribute_dataset(val_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)


wbargs = {
    'Batch Size': GLOBAL_BATCH_SIZE,
    'Notes':"tf_v100_8x",
    'Tags': ['Multi', 'gpu', 'tensorflow'],
    'dataset':"MNIST",
    'architecture':"Custom"}


# Open a strategy scope.
with strategy.scope():
    model = get_compiled_model(base_lr)



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Found 3068 files belonging to 10 classes.
Found 341 files belonging to 10 classes.
Found 372 files belonging to 10 classes.


In [5]:
3068/64

47.9375

In [9]:
# --------- Start wandb --------- #
wandb.init(entity='skirmer', project='tensorflow_blog')
config=wbargs


start = time.time()
model.fit(train_dist_dataset, 
          steps_per_epoch =  3067/100,
          epochs=50, 
          validation_data=val_dataset, 
          callbacks=[WandbCallback()])
end = time.time()-start
print("model training time", end)
wandb.log({"training_time":end})

# Test the model on all available devices.
model.evaluate(test_dataset)

# Close your wandb run 
wandb.run.finish()

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
30/30 [==============================] - 85s 441ms/step - loss: 9.2617 - accuracy: 0.1579 - val_loss: 5238710561713160192.0000 - val_accuracy: 0.0733
Epoch 2/50
30/30 [==============================] - 6s 212ms/step - loss: 3.2500 - accuracy: 0.2419 - val_loss: 6317135872.0000 - val_accuracy: 0.0733
Epoch 3/50
30/30 [==============================] - 6s 211ms/step - loss: 2.2024 - accuracy: 0.2512 - val_loss: 552813.8750 - val_accuracy: 0.0733
Epoch 4/50
30/30 [==============================] - 6s 212ms/step - loss: 2.1312 - accuracy: 0.2627 - val_loss: 1744.7021 - val_accuracy: 0.0821
Epoch 5/50
30/30 [==============================] - 6s 210ms/step - loss: 2.0331 - accuracy: 0.2999 - val_loss: 86.5837 - val_accuracy: 0.1320
Epoch 6/50
30/30 [==============================] - 6s 209ms/step - loss: 1.949

Exception ignored in: <function ScopedTFFunction.__del__ at 0x7f8b95eb94c0>
Traceback (most recent call last):
  File "/srv/conda/envs/saturn/lib/python3.9/site-packages/tensorflow/python/framework/c_api_util.py", line 104, in __del__
    def __del__(self):
KeyboardInterrupt: 


30/30 [==============================] - 7s 231ms/step - loss: 1.2879 - accuracy: 0.5537 - val_loss: 3.0738 - val_accuracy: 0.2375
Epoch 19/50
18/30 [================>.............] - ETA: 2s - loss: 1.3877 - accuracy: 0.5131

KeyboardInterrupt: 